In [352]:
import pickle
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import xgboost as xgb

In [353]:
f = open("train_data.pkl", "rb")
train_data = pickle.load(f)
f = open("test_data.pkl", "rb")
test_data = pickle.load(f)

In [354]:
X = train_data.drop(['OC', 'inst_id'], axis=1)
y = train_data.loc[:, train_data.columns == 'OC']
X_test2 = test_data.drop(['OC', 'inst_id'], axis=1)

In [355]:
from sklearn.model_selection import train_test_split

In [372]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3)

In [373]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.5, nthread=7)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [374]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.978021978021978

In [375]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [376]:
param = {'max_depth': 2, 'eta': 0.5, 'silent': 1, 'objective': 'binary:logistic'}
param['nthread'] = 7
param['eval_metric'] = 'auc'
evallist = [(dtest, 'eval'), (dtrain, 'train')]
plst = param.items()

In [377]:
num_round = 50
bst = xgb.train(plst, dtrain, num_round, evallist)

[0]	eval-auc:0.75	train-auc:0.685474
[1]	eval-auc:0.926966	train-auc:0.923467
[2]	eval-auc:0.991573	train-auc:0.928934
[3]	eval-auc:0.988764	train-auc:0.939282
[4]	eval-auc:0.966292	train-auc:0.976572
[5]	eval-auc:0.94382	train-auc:0.984772
[6]	eval-auc:0.938202	train-auc:0.989262
[7]	eval-auc:0.94382	train-auc:0.994533
[8]	eval-auc:0.921348	train-auc:0.995705
[9]	eval-auc:0.921348	train-auc:0.997462
[10]	eval-auc:0.904494	train-auc:0.998438
[11]	eval-auc:0.88764	train-auc:0.999024
[12]	eval-auc:0.91573	train-auc:0.999805
[13]	eval-auc:0.893258	train-auc:0.999805
[14]	eval-auc:0.926966	train-auc:0.999414
[15]	eval-auc:0.921348	train-auc:0.999414
[16]	eval-auc:0.94382	train-auc:0.999414
[17]	eval-auc:0.926966	train-auc:0.999805
[18]	eval-auc:0.921348	train-auc:0.999805
[19]	eval-auc:0.926966	train-auc:0.999805
[20]	eval-auc:0.921348	train-auc:0.999805
[21]	eval-auc:0.904494	train-auc:0.999805
[22]	eval-auc:0.904494	train-auc:0.999805
[23]	eval-auc:0.893258	train-auc:0.999805
[24]	eval-a

In [378]:
ypred = bst.predict(dtest, ntree_limit=bst.best_ntree_limit)

In [379]:
(sum((ypred>0.5) == (np.array(y_test)).flat)) / 91

0.978021978021978

In [380]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [386]:
estimator = XGBClassifier(n_estimators=1000, max_depth=2, learning_rate=0.5, nthread=7)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.05, 0.5, 1],
    'n_estimators': [200,300,500,1000],
    'max_depth' : [4,5,6]
}

gbm = GridSearchCV(estimator, param_grid, cv=5, scoring='roc_auc')

gbm.fit(X, y)

print('Best parameters found by grid search are:', gbm.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.01, 'max_depth': 5, 'n_estimators': 300}


In [391]:
answer = gbm.best_estimator_.predict(X_test2)
answer = pd.DataFrame(answer)
result = pd.concat([X_test2, test_data['inst_id'], answer], axis=1)
result.to_csv("answer2.csv")